In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle
import numpy as np
from tensorflow.keras.callbacks import TensorBoard
import time


# tf.debugging.set_log_device_placement(True)
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
NAME= "songs-cnn-noreshape-32conv-32den-{}".format(int(time.time()))

tensorboard = TensorBoard(log_dir='songlogs/{}'.format(NAME))

X = pickle.load(open("/content/drive/MyDrive/ColabNotebooks/lab_data_songsX.pickle","rb"))
y = pickle.load(open("/content/drive/MyDrive/ColabNotebooks/lab_data_songsy.pickle","rb"))

X = X/255.0

# print((X[0][0]))
y = np.array(y)





# New Section

In [ ]:
x_min = X.min(axis=(1, 2), keepdims=True)
x_max = X.max(axis=(1, 2), keepdims=True)
X = (X - x_min)/(x_max-x_min)

In [ ]:
print(np.max(X[1]))
print(np.min(X[1]))

1.0
0.0


In [ ]:


dense_layers = [0,1,2] # was 0, 1, 2
layer_sizes = [32, 64, 128] # was 32, 64, 128
conv_layers = [1, 2, 3] #was 1, 2, 3

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            model.compile(loss='binary_crossentropy',
                          optimizer='Adam',
                          metrics=['accuracy'],
                          )

            model.fit(X, y,
                      batch_size=32,
                      epochs=10,
                      validation_split=0.2,
                      callbacks=[tensorboard])

3-conv-128-nodes-2-dense-1697305084
Epoch 1/10
932/932 [==============================] - 109s 111ms/step - loss: 0.2203 - accuracy: 0.9104 - val_loss: 0.1155 - val_accuracy: 0.9581
Epoch 2/10
932/932 [==============================] - 99s 106ms/step - loss: 0.0937 - accuracy: 0.9682 - val_loss: 0.0867 - val_accuracy: 0.9724
Epoch 3/10
932/932 [==============================] - 99s 106ms/step - loss: 0.0605 - accuracy: 0.9785 - val_loss: 0.1040 - val_accuracy: 0.9677
Epoch 4/10
932/932 [==============================] - 99s 106ms/step - loss: 0.0407 - accuracy: 0.9856 - val_loss: 0.1083 - val_accuracy: 0.9647
Epoch 5/10
932/932 [==============================] - 99s 106ms/step - loss: 0.0271 - accuracy: 0.9906 - val_loss: 0.0906 - val_accuracy: 0.9792
Epoch 6/10
932/932 [==============================] - 98s 106ms/step - loss: 0.0220 - accuracy: 0.9928 - val_loss: 0.1170 - val_accuracy: 0.9712
Epoch 7/10
932/932 [==============================] - 98s 105ms/step - loss: 0.0164 - accurac

In [ ]:
model.save("oct14_98_lab_songs.model")